In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
!ls results-1024

checkpoint-1000   checkpoint-13000  checkpoint-4500  checkpoint-7500
checkpoint-10000  checkpoint-1500   checkpoint-500   checkpoint-8000
checkpoint-10500  checkpoint-2000   checkpoint-5000  checkpoint-8500
checkpoint-11000  checkpoint-2500   checkpoint-5500  checkpoint-9000
checkpoint-11500  checkpoint-3000   checkpoint-6000  checkpoint-9500
checkpoint-12000  checkpoint-3500   checkpoint-6500  final_merged_checkpoint
checkpoint-12500  checkpoint-4000   checkpoint-7000  runs


In [3]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

2023-07-30 15:53:30.375454: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 15:53:30.472207: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-30 15:53:31.094040: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-30 15:53:31.094097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [4]:
import torch

In [5]:
model = AutoPeftModelForCausalLM.from_pretrained(
    './results-1024/checkpoint-13000', device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)

In [7]:
model.device

device(type='cuda', index=0)

In [8]:
template = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [9]:
query = """
paragraph `KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.

Ketua Pemuda Pas Terengganu, Mohd Harun Esa berkata, isu tersebut telah reda dan kembali dibangkitkan supaya rakyat memandang serong terhadap parti berkenaan.

"Ia hanyalah tindakan politik dan terdesak pihak kerajaan yang ketandusan isu untuk mengaitkan pembangkang dengan kesalahan.

"Isu ini sudah lama dan sudah reda namun seperti mereka ini (kerajaan) masih dengan mentaliti 'pembangkang' kerana menghangatkan sesuatu isu supaya rakyat pandang serong kepada PN," katanya ketika dihubungi Sinar Harian pada Isnin.`

extract all entities in the paragraph, bagi dalam JSON
"""
prompt = template['prompt_no_input'].format(instruction=query)
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
input_ids

tensor([[    1,  4671,   289,  1450,   801,   474,   284,   801, 24418,  5403,
           343,   574,   286,   759,   574, 11052,   260,   688,   294,   273,
         29889, 27415,   275,   432,  1450, 21419,   343,   574,  3999, 29884,
          1794,   972,  6249, 24418,  5403,  1935,   344,  4187, 29889,    13,
            13,  2277, 29937, 25953,  5403, 29901,    13,    13, 26956,   421,
         29968, 29965,  1964, 29909,   323,  1001,  1430, 26788,  2190, 29965,
           448, 12693,  9919,   273,  1258,   557,  2766,   413,   300,   392,
           375,   273,   338, 29884,   409,  2790,  3249,   286,   996,  1249,
         11052,   282,   331,  6574,   262,  2431,   638, 23402, 22318,  1848,
           313, 15695, 29897,   972,  6249,   413, 15274,   532,   273,  7655,
          6948,   332,   639,  1335,  7889, 29889,    13,    13, 29968,   300,
          3357,   349,   331,  6191, 10043,  5061,   996,  6249, 29884, 29892,
         12929, 29881,  3536,   348,  3423, 29874,  

In [10]:
max_new_tokens = 300
temperature = 0.9
top_p = 0.95
top_k = 50
num_beams = 1
do_sample = True
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=do_sample,
)

In [11]:
tokenizer.decode(generation_output.sequences[0])

'<s> Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n\nparagraph `KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.\n\nKetua Pemuda Pas Terengganu, Mohd Harun Esa berkata, isu tersebut telah reda dan kembali dibangkitkan supaya rakyat memandang serong terhadap parti berkenaan.\n\n"Ia hanyalah tindakan politik dan terdesak pihak kerajaan yang ketandusan isu untuk mengaitkan pembangkang dengan kesalahan.\n\n"Isu ini sudah lama dan sudah reda namun seperti mereka ini (kerajaan) masih dengan mentaliti \'pembangkang\' kerana menghangatkan sesuatu isu supaya rakyat pandang serong kepada PN," katanya ketika dihubungi Sinar Harian pada Isnin.`\n\nextract all entities in the paragraph, bagi dalam JSON\n\n\n### Jawapan:\n{\n  "name": "Kerajaan",\n  "entity_types": []\n}\n\n{\n  "name": "Mohd Harun Esa",\n  "entity_types": [\n    "person"\n  

In [12]:
output_merged_dir = './results-1024/final_merged_checkpoint'
model.save_pretrained(output_merged_dir, safe_serialization=True, )

In [13]:
!ls {output_merged_dir}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
config.json			  model-00002-of-00002.safetensors
generation_config.json		  model.safetensors.index.json
model-00001-of-00002.safetensors


In [14]:
model.push_to_hub('llama-7b-hf-1024-ms-qlora', organization='mesolitica')

/home/husein/.local/lib/python3.8/site-packages/transformers/utils/hub.py:665: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/llama-7b-hf-1024-ms-qlora/commit/9d18f692d21f93a510ee09dae62d3147d0429b3c', commit_message='Upload LlamaForCausalLM', commit_description='', oid='9d18f692d21f93a510ee09dae62d3147d0429b3c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('llama-7b-hf-1024-ms-qlora', organization='mesolitica')